### Openradar.io Historical Image Generator

Please follow these steps to running this script

1. Edit the configuration below. Stick to the basic options if you're unsure of the rest
1. From the top menus select: 'Cell' -> 'Run All'
1. A download link will print at the bottom of this notebook when the images are ready

In [1]:
# BASIC CONFIGURATION
rid   = '50'         # radar identification number (leading zero needed for single digits), string
rdate = '2014/11/27' # date to start the image generation (format: YYYY/MM/DD), string
rtime = '15:00'      # time to start the image generation (format: HH:MM), string
rdur  = 0.5            # number of hours from start date/time for image generation (MUST NOT EXTEND TO THE NEXT DAY), integer
prng  = 150          # number of kilometers from the radar site to plot, integer
field = 'reflectivity'       # product to plot. Reflectivity = 'reflectivity', Doppler Velocity = 'velocity'
                             # (Others/Polarimetric:differential_reflectivity, cross_correlation_ratio, specific_differential_phase, differential_phase, spectrum_width), string
# MAP CONFIGURATION
hide_place_names = [''] #list of placenames you wish to hide from the map (they are auto generated), list of string
nice_radar_name = 'Marburg' #name of radar to use for title, string
nice_colorbar_name = field #name to use for colorbar
range_ring_list = [50, 100] #list of range ring disances from the radar site (km), list of numbers

# ADVANCED CONFIGURATION
tilt     = 1                      # PPI index in volume to plot, first/lowest PPI is number 0, integer
# ADVANCED CONFIGURATION - Reflectivity
vmin_ref = 20                     # minimum of colourmap (values less than will be removed), integer
vmax_ref = 65                     # maximum of colormap (values greater than will be removed), integer
cmap_ref = 'pyart_HomeyerRainbow' # colormap to use. Another option is 'pyart_NWSRef' for more see https://arm-doe.github.io/pyart/API/generated/pyart.graph.html, string
# ADVANCED CONFIGURATION - Doppler Velocity
vmin_vel =-26                     # minimum of colormap, integer
vmax_vel = 26                     # maximum of colormap, integer
cmap_vel = 'pyart_BuDRd18'        # colormap to use, string
# ADVANCED CONFIGURATION - Other/Polarimetric
vmin_pol = -1                     # minimum of colormap, integer
vmax_pol = 5                      # maximum of colormap, integer
cmap_pol = 'pyart_HomeyerRainbow' # olormap to use, string

In [2]:
import core

cdict = {
    'rid':rid, 'rdate':rdate, 'rtime':rtime, 'rdur':rdur, 'prng':prng, 'field':field,
    'hide_place_names':hide_place_names,'nice_radar_name':nice_radar_name, 'range_ring_list':range_ring_list,
    'tilt':tilt,
    'vmin_ref':vmin_ref, 'vmax_ref':vmax_ref, 'cmap_ref':cmap_ref,
    'vmin_vel':vmin_vel, 'vmax_vel':vmax_vel, 'cmap_vel':cmap_vel,
    'vmin_pol':vmin_pol, 'vmax_pol':vmax_pol, 'cmap_pol':cmap_pol
        }

core.build_animation(cdict)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

Fetching: http://dapds00.nci.org.au/thredds/fileServer/rq0/50/2014/vol/50_20141127.pvol.zip


  0%|          | 0/3 [00:00<?, ?it/s]

Generating Images


100%|██████████| 3/3 [00:08<00:00,  3.00s/it]


/home/meso/dev/openradar-binder/images/50_2014-11-27.image_request.16-04-2020_12:33:19.zip